In [6]:
!pip3 install torchvision

In [7]:
import cv2
import os
import zipfile
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np




In [8]:
extract_dir='/Users/apoorvagayatrik/PerspectAIProj/dataset'

In [9]:
import os
import glob
import cv2
import numpy as np
import torch

os.chdir('/Users/apoorvagayatrik/PerspectAIProj/ESRGAN')
import RRDBNet_arch as arch

# Define the apply_esrgan function
def apply_esrgan(img, model, device):
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round().astype(np.uint8)

    return output

In [10]:
# Load the ESRGAN model
model_path = '/Users/apoorvagayatrik/PerspectAIProj/ESRGAN/models/RRDB_ESRGAN_x4.pth'
device = torch.device('cpu')  # if you want to run on CPU, change 'cuda' -> 'cpu'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Example for a generic model
total_params = count_parameters(model)
print(f'Total number of parameters: {total_params}')

def get_model_size(model, filepath):
    torch.save(model.state_dict(), filepath)  # For PyTorch, or appropriate save function for other frameworks
    return os.path.getsize(filepath)

model_size = get_model_size(model, 'model.pth')
print(f'Model size: {model_size / (1024**2)} MB')






Total number of parameters: 16697987
Model size: 63.88340950012207 MB


In [11]:
def detect_faces(image):

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    return faces

In [7]:
datasets=['train_ds','val_ds','test_ds']

train_preprocessed=[]
val_preprocessed=[]
test_preprocessed=[]


In [8]:
!pip3 install tqdm

In [9]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Assuming detect_faces and apply_esrgan functions are already defined

datasets = ['train_ds', 'val_ds', 'test_ds']
extract_dir = '/Users/apoorvagayatrik/PerspectAIProj/dataset'  # Adjust this to your actual extract directory
output_dir = '/Users/apoorvagayatrik/PerspectAIProj/processed_images'  # Directory to save processed images

# Initialize lists for storing preprocessed images
train_preprocessed = []
val_preprocessed = []
test_preprocessed = []

train_labels = []
val_labels = []
test_labels = []

def load_images_and_labels(dataset):
    images = []
    labels = []
    dataset_path = os.path.join(extract_dir, dataset)
    people_folders = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]

    progress_bar = tqdm(total=len(people_folders), desc=f'Processing {dataset}', unit='folder')

    for person_folder in people_folders:
        person_path = os.path.join(dataset_path, person_folder)
        image_files = [os.path.join(person_path, f) for f in os.listdir(person_path) if f.endswith('.jpg') or f.endswith('.png')]
        label = person_folder
        for image_path in image_files:
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            faces = detect_faces(image)

            for (x, y, w, h) in faces:
                # Extract the face ROI
                face_roi = image[y:y+h, x:x+w]
                # Apply ESRGAN to the face ROI
                enhanced_face = apply_esrgan(face_roi, model, device)

                # Convert to grayscale
                gray_face = cv2.cvtColor(enhanced_face, cv2.COLOR_BGR2GRAY)

                # Resize the face to the target size
                target_size = (64, 64)
                resized_face = cv2.resize(gray_face, target_size)

                # Save processed image
                filename = os.path.basename(image_path)
                output_path = os.path.join(output_dir, dataset, person_folder, filename)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                cv2.imwrite(output_path, resized_face)

                images.append(resized_face)
                labels.append(label)
        
        progress_bar.update(1)

    progress_bar.close()
    return images, labels

# Call the function to load and process images
train_preprocessed, train_labels = load_images_and_labels('train_ds')
val_preprocessed, val_labels = load_images_and_labels('val_ds')
test_preprocessed, test_labels = load_images_and_labels('test_ds')

print("Image loading and enhancement completed!")


Processing test_ds: 100%|████████████████| 35/35 [5:45:31<00:00, 592.33s/folder]

Image loading and enhancement completed!


In [2]:
import torch

# Check if CUDA (GPU acceleration) is available
cuda_available = torch.cuda.is_available()

if cuda_available:
    # Get the CUDA device count
    cuda_device_count = torch.cuda.device_count()
    print('CUDA is available with {cuda_device_count} CUDA device(s)!')
else:
    print('CUDA is not available. Running on CPU.')


CUDA is not available. Running on CPU.


In [26]:
def load_images_and_labels_from_directory(base_dir, target_size=(64, 64)):
    images = []
    labels = []
    for label in os.listdir(base_dir):
        label_dir = os.path.join(base_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                if filename.endswith('.png') or filename.endswith('.jpg'):
                    img_path = os.path.join(label_dir, filename)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        resized_img = cv2.resize(img, target_size)
                        images.append(resized_img)
                        labels.append(label)
    return np.array(images), np.array(labels)

In [27]:
train_images, train_labels = load_images_and_labels_from_directory('/Users/apoorvagayatrik/PerspectAIProj/processed_images/train_ds')
val_images, val_labels = load_images_and_labels_from_directory('/Users/apoorvagayatrik/PerspectAIProj/processed_images/val_ds')
test_images, test_labels = load_images_and_labels_from_directory('/Users/apoorvagayatrik/PerspectAIProj/processed_images/test_ds')


In [32]:
def compute_covmatrix(images):
    num_images = len(images)
    flattened_images = np.array([img.flatten() for img in images])
    mean_image = np.mean(flattened_images, axis=0)
    cov_matrix = np.cov(flattened_images, rowvar=False)
    return cov_matrix



train_cov_matrix = compute_covmatrix(train_images)
val_cov_matrix = compute_covmatrix(val_images)
test_cov_matrix = compute_covmatrix(test_images)

In [33]:
def extract_principal_components(cov_matrix,num_components):
    eig_vals,eig_vecs=np.linalg.eigh(cov_matrix)

    sorted_indices=np.argsort(eig_vals)[::-1]
    sorted_eig_vals=eig_vals[sorted_indices]
    sorted_eig_vecs=eig_vecs[:,sorted_indices]
  

    principal_components=sorted_eig_vecs[:,:num_components]

    return principal_components


num_components = 50

train_principal_components = extract_principal_components(train_cov_matrix, num_components)
val_principal_components = extract_principal_components(val_cov_matrix, num_components)
test_principal_components = extract_principal_components(test_cov_matrix, num_components)

In [34]:
def project_images(images,principal_components):
    num_images = len(images)
    flattened_images = images.reshape(num_images, -1)  # Flatten each image into a vector
    projected_images = np.dot(flattened_images, principal_components)

    return projected_images

train_projected = project_images(train_images, train_principal_components)


In [37]:
from sklearn.neighbors import KNeighborsClassifier

def train_classifier(projections,labels):
    knn=KNeighborsClassifier(n_neighbors=5)
    knn.fit(projections,labels)
    return knn


knn_classifier = train_classifier(train_projected, train_labels)


In [36]:
!pip3 install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 502.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 9.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 10.2 MB/s eta 0:00:0000:0100:01


In [45]:
from sklearn.metrics import accuracy_score, classification_report


def evaluate_classifier(knn_classifier, test_images, test_labels, principal_components):
    num_images = len(test_images)
    flattened_images = np.array([img.flatten() for img in test_images])
    projected_images = np.dot(flattened_images, principal_components)
    predictions = knn_classifier.predict(projected_images)
    accuracy = accuracy_score(test_labels, predictions)
    report = classification_report(test_labels, predictions)
    return accuracy, report

# Evaluate classifier on validation and test datasets
val_accuracy, val_report = evaluate_classifier(knn_classifier, val_images, val_labels, val_principal_components)
test_accuracy, test_report = evaluate_classifier(knn_classifier, test_images, test_labels, test_principal_components)
print(f"Validation Accuracy: {val_accuracy}")
print(f"Validation Report:\n{val_report}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Report:\n{test_report}")



Validation Accuracy: 0.0
Validation Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
          10       0.00      0.00      0.00       0.0
        1003       0.00      0.00      0.00       0.0
        1013       0.00      0.00      0.00       0.0
        1017       0.00      0.00      0.00       0.0
        1025       0.00      0.00      0.00       0.0
        1027       0.00      0.00      0.00       0.0
        1029       0.00      0.00      0.00       0.0
        1032       0.00      0.00      0.00       0.0
        1036       0.00      0.00      0.00       0.0
        1038       0.00      0.00      0.00       0.0
         104       0.00      0.00      0.00       0.0
        1040       0.00      0.00      0.00       0.0
        1042       0.00      0.00      0.00       0.0
        1044       0.00      0.00      0.00       0.0
        1045       0.00      0.00      0.00       0.0
        1049       0.00      0.00    

/Users/apoorvagayatrik/miniforge3/envs/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/apoorvagayatrik/miniforge3/envs/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/apoorvagayatrik/miniforge3/envs/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi

In [44]:
def recognize_faces(classifier, images, principal_components):
    num_images = len(images)
    flattened_images = np.array([img.flatten() for img in images])
    projected_images = np.dot(flattened_images, principal_components)
    predictions = classifier.predict(projected_images)
    return predictions


# Perform face recognition on test set
test_predictions = recognize_faces(knn_classifier, test_images, train_principal_components)

In [ ]:
model_path = '/Users/apoorvagayatrik/PerspectAIProj/ESRGAN/models/RRDB_ESRGAN_x4.pth'


model_size = get_model_size(model, model_path)

total_params = count_parameters(model)
print('Total number of parameters: {total_params}')
